In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%%bash
cd /content
if [ ! -d "ComfyUI" ]; then
  git clone https://github.com/comfyanonymous/ComfyUI.git
else
  cd ComfyUI && git pull
fi


Cloning into 'ComfyUI'...


In [3]:
%%bash
DRIVE_BASE="/content/drive/MyDrive/Fooocus"
RUNTIME="/content/ComfyUI"

mkdir -p "$DRIVE_BASE/models/checkpoints"

ln -sfn "$DRIVE_BASE/models/checkpoints" "$RUNTIME/models/checkpoints"
ln -sfn "$DRIVE_BASE/models/loras" "$RUNTIME/models/loras"

echo "=== Checkpoints in Drive ==="
ls -lh "$DRIVE_BASE/models/checkpoints"


=== Checkpoints in Drive ===
total 29G
-rw------- 1 root root  23G Aug 18 20:53 fuxCapacityNSFWPornFlux_31FP16.safetensors
-rw------- 1 root root 6.7G Aug 17 11:53 juggernautXL_v8Rundiffusion.safetensors


In [4]:
%%bash
cd /content/ComfyUI
pip install -q torch torchvision torchaudio xformers --extra-index-url https://download.pytorch.org/whl/cu118
pip install -q -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 119.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
!python /content/ComfyUI/main.py --listen --port 7860 --share

usage: main.py [-h] [--listen [IP]] [--port PORT] [--tls-keyfile TLS_KEYFILE]
               [--tls-certfile TLS_CERTFILE] [--enable-cors-header [ORIGIN]]
               [--max-upload-size MAX_UPLOAD_SIZE]
               [--base-directory BASE_DIRECTORY]
               [--extra-model-paths-config PATH [PATH ...]]
               [--output-directory OUTPUT_DIRECTORY]
               [--temp-directory TEMP_DIRECTORY]
               [--input-directory INPUT_DIRECTORY] [--auto-launch]
               [--disable-auto-launch] [--cuda-device DEVICE_ID]
               [--default-device DEFAULT_DEVICE_ID]
               [--cuda-malloc | --disable-cuda-malloc]
               [--force-fp32 | --force-fp16]
               [--fp32-unet | --fp64-unet | --bf16-unet | --fp16-unet | --fp8_e4m3fn-unet | --fp8_e5m2-unet | --fp8_e8m0fnu-unet]
               [--fp16-vae | --fp32-vae | --bf16-vae] [--cpu-vae]
               [--fp8_e4m3fn-text-enc | --fp8_e5m2-text-enc | --fp16-text-enc | --fp32-text-enc | --bf1

In [11]:
# 1) Settings
COMFY_DIR = "/content/ComfyUI"
PORT = 7860
NGROK_TOKEN = "31VTKtzAuQW2Be4u6HUb6fpNA6o_T1ET7QyEQmCkyaTSiAHn"  # <-- paste your token

# 2) Imports & installs
import subprocess, time, socket, os, sys
!pip -q install pyngrok

# 3) Start ComfyUI first (background)
#    Bind to 0.0.0.0 and chosen port; capture logs
log_path = "/content/comfy.log"
if os.path.exists(log_path):
    os.remove(log_path)

comfy = subprocess.Popen(
    ["python3", "main.py", "--listen", "0.0.0.0", "--port", str(PORT)],
    cwd=COMFY_DIR,
    stdout=open(log_path, "w"),
    stderr=subprocess.STDOUT,
    text=True,
)

# 4) Wait until port is open (or report crash)
def is_port_open(host, port):
    with socket.socket() as s:
        s.settimeout(1.0)
        try:
            s.connect((host, port))
            return True
        except Exception:
            return False

print("Starting ComfyUI… waiting for port", PORT)
deadline = time.time() + 120  # wait up to 120s
while time.time() < deadline:
    if comfy.poll() is not None:
        print("ComfyUI process exited early. Last 100 log lines:\n")
        print("".join(open(log_path).readlines()[-100:]))
        raise SystemExit(1)
    if is_port_open("127.0.0.1", PORT):
        print("ComfyUI is up on 127.0.0.1:%d" % PORT)
        break
    time.sleep(2)
else:
    print("Timed out waiting for ComfyUI. Last 100 log lines:\n")
    print("".join(open(log_path).readlines()[-100:]))
    raise SystemExit(1)

# 5) Open ngrok tunnel AFTER ComfyUI is ready
from pyngrok import ngrok, conf
conf.get_default().auth_token = NGROK_TOKEN
public_url = ngrok.connect(PORT, "http")
print("🌍 Public ComfyUI URL:", public_url)

# 6) Tail logs a bit so you can see activity
print("\n--- Last 30 log lines ---")
print("".join(open(log_path).readlines()[-30:]))


Starting ComfyUI… waiting for port 7860
ComfyUI is up on 127.0.0.1:7860
🌍 Public ComfyUI URL: NgrokTunnel: "https://6f7fb70d58f6.ngrok-free.app" -> "http://localhost:7860"

--- Last 30 log lines ---

